In [7]:
import tensorflow.keras
from keras.models import Sequential
from keras.layers import Dense,SimpleRNN,Embedding
import sklearn
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import numpy
from numpy import array 


In [2]:
data="In modern football we are in the After Kaka  era .\n where conversations about the best in the world evolved from a list of many to a shortlist of two .\n The game that Messi and Ronaldo have played in the last 10 years is not the same one in which Kaka excelled .\n The duo moved the sport onwards and demanded everyone else come with them."
data=data.split('\n')
tk=Tokenizer(filters='')# if filter = '$%' then it will exclude and not consider as token .here in data there is no special char other then . and we want it so it is null 
tk.fit_on_texts(data) #making the vocab
print("vocab is :\n",tk.word_index)
vocab_len=len(tk.word_index)+1 # extra one for handling unknown token
print("vocab length: ",vocab_len)

enSeq=tk.texts_to_sequences(data)
print("encoded sentences are:\n",enSeq)
X=[]
Y=[]

for i in range(len(enSeq)):
  X.insert(i,enSeq[i][:-1]) # we do not need '.' in X becuse in last unfolding we only give input till word before '.' 
  Y.insert(i,enSeq[i])

print("input:",X)
print("output:",Y)

"""for i in range(len(X)):
  #l=0 + X[i]
  X[i].insert(0,0)
print("adding 0 in training:\n",X)"""

maxlen=max([len(s) for s in X ])
X=pad_sequences(X,maxlen=maxlen+1,padding='pre')
print("AFTER padding:",X,"shape:\n",X.shape)
Y=pad_sequences(Y,maxlen=maxlen+1,padding='pre')

Y=np_utils.to_categorical(Y,num_classes=len(tk.word_index)+1)
print("one hot vector y\n",Y)
print(X.shape,Y.shape)

vocab is :
 {'the': 1, 'in': 2, '.': 3, 'kaka': 4, 'a': 5, 'of': 6, 'and': 7, 'modern': 8, 'football': 9, 'we': 10, 'are': 11, 'after': 12, 'era': 13, 'where': 14, 'conversations': 15, 'about': 16, 'best': 17, 'world': 18, 'evolved': 19, 'from': 20, 'list': 21, 'many': 22, 'to': 23, 'shortlist': 24, 'two': 25, 'game': 26, 'that': 27, 'messi': 28, 'ronaldo': 29, 'have': 30, 'played': 31, 'last': 32, '10': 33, 'years': 34, 'is': 35, 'not': 36, 'same': 37, 'one': 38, 'which': 39, 'excelled': 40, 'duo': 41, 'moved': 42, 'sport': 43, 'onwards': 44, 'demanded': 45, 'everyone': 46, 'else': 47, 'come': 48, 'with': 49, 'them.': 50}
vocab length:  51
encoded sentences are:
 [[2, 8, 9, 10, 11, 2, 1, 12, 4, 13, 3], [14, 15, 16, 1, 17, 2, 1, 18, 19, 20, 5, 21, 6, 22, 23, 5, 24, 6, 25, 3], [1, 26, 27, 28, 7, 29, 30, 31, 2, 1, 32, 33, 34, 35, 36, 1, 37, 38, 2, 39, 4, 40, 3], [1, 41, 42, 1, 43, 44, 7, 45, 46, 47, 48, 49, 50]]
input: [[2, 8, 9, 10, 11, 2, 1, 12, 4, 13], [14, 15, 16, 1, 17, 2, 1, 18, 19

In [3]:
#making the model

model=Sequential()
model.add(Embedding(input_dim=vocab_len,output_dim=10))
print(model.input_shape)
print(model.output_shape)

model.add(SimpleRNN(50,return_sequences=True)) # out put of previous fold will be used in input
print(model.output_shape)
model.add(Dense(units=vocab_len,activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

(None, None)
(None, None, 10)
(None, None, 50)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 10)          510       
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, None, 50)          3050      
_________________________________________________________________
dense (Dense)                (None, None, 51)          2601      
Total params: 6,161
Trainable params: 6,161
Non-trainable params: 0
_________________________________________________________________


In [4]:
model.fit(X,Y,epochs=500,verbose=1)

Epoch 1/500
1/1 [==============================] - 4s 4s/step - loss: 3.9298 - accuracy: 0.0109
Epoch 2/500
1/1 [==============================] - 0s 20ms/step - loss: 3.8963 - accuracy: 0.2717
Epoch 3/500
1/1 [==============================] - 0s 17ms/step - loss: 3.8658 - accuracy: 0.3370
Epoch 4/500
1/1 [==============================] - 0s 16ms/step - loss: 3.8285 - accuracy: 0.3478
Epoch 5/500
1/1 [==============================] - 0s 19ms/step - loss: 3.7833 - accuracy: 0.3261
Epoch 6/500
1/1 [==============================] - 0s 16ms/step - loss: 3.7401 - accuracy: 0.3370
Epoch 7/500
1/1 [==============================] - 0s 14ms/step - loss: 3.7307 - accuracy: 0.2935
Epoch 8/500
1/1 [==============================] - 0s 14ms/step - loss: 3.6694 - accuracy: 0.3261
Epoch 9/500
1/1 [==============================] - 0s 14ms/step - loss: 3.6190 - accuracy: 0.3261
Epoch 10/500
1/1 [==============================] - 0s 14ms/step - loss: 3.5754 - accuracy: 0.3261
Epoch 11/500
1/1 [===

In [8]:
# Sentance sampling generating all words without seed considering highest probability word as next word
def sample_sequence(model_RNN, courpus_tokenizer, words, size_vocab):
  enter = list()
  text_input = ''
  for i in range(words):
    enter = courpus_tokenizer.texts_to_sequences([text_input])[0]
    enter.insert(0,0)
    enter = array(enter)
    enter = numpy.reshape(enter, newshape=(1,-1))
    print("i:", i, "Enter:", enter, enter.shape)

    if i==0:
      probability = model_RNN.predict_proba(enter, verbose=0)
      print("i:", i, "Probability:", probability, type(probability), probability.shape)
      y_predict = 0
      while y_predict==0:
        y_predict = numpy.random.choice(range(size_vocab), p = probability.ravel())
      y_predict = numpy.reshape(numpy.array([y_predict]), newshape = (1,-1))
      print("i:", i, "If Y_Predict:", y_predict, y_predict.shape)
    
    else:
      probability = model_RNN.predict_proba(enter, verbose=0)
      print("i:", i, "Probability:", probability, type(probability), probability.shape)
      y_predict = numpy.append(y_predict,0)
      y_predict = numpy.reshape(y_predict, newshape = (1,-1))

      while y_predict[0,i]==0:
        y_predict[0,i]=numpy.random.choice(range(size_vocab),p=probability[0,i].ravel())
      print("i:", i, "Else Y_Predict:", y_predict, y_predict.shape)

    
    exit = ''
    for w, index in courpus_tokenizer.word_index.items():
      if index == y_predict[0,i]:
        exit = w
        print("Index:", i, "Word:", w)
        break

    
    text_input = str(text_input) + str(exit) + ' '
  return text_input




print(sample_sequence(model, tk, 8, vocab_len))
    

i: 0 Enter: [[0]] (1, 1)
i: 0 Probability: [[[7.46949375e-01 2.46793196e-01 1.83577335e-03 1.88097212e-04
   4.71277017e-05 3.81900812e-04 1.91070603e-06 2.09650784e-06
   7.35414214e-04 1.49924992e-04 1.42769670e-04 1.34450411e-05
   4.17966512e-05 3.41359519e-05 7.06000603e-04 1.92947791e-05
   4.74016197e-05 5.33839884e-05 2.16858425e-05 5.09072561e-05
   3.84136947e-05 4.94236156e-05 7.20680109e-05 1.75826626e-05
   1.11210880e-04 1.13037415e-04 1.09643435e-04 4.92015934e-05
   2.30149981e-05 5.09998790e-05 8.83653629e-05 6.67178319e-05
   3.35102163e-06 1.27836154e-06 1.49351672e-05 1.40283109e-05
   8.95757112e-05 1.54491372e-05 1.04128580e-04 1.66033060e-04
   1.47720042e-04 1.81505911e-05 6.44067995e-06 3.05070189e-05
   5.64049005e-05 1.59723140e-06 4.23822576e-06 1.75711302e-05
   1.56284441e-05 2.06641329e-04 8.10355850e-05]]] <class 'numpy.ndarray'> (1, 1, 51)
i: 0 If Y_Predict: [[1]] (1, 1)
Index: 0 Word: the
i: 1 Enter: [[0 1]] (1, 2)


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


i: 1 Probability: [[[7.46949434e-01 2.46793106e-01 1.83577160e-03 1.88097227e-04
   4.71277053e-05 3.81900638e-04 1.91070603e-06 2.09650602e-06
   7.35413807e-04 1.49924876e-04 1.42769539e-04 1.34450293e-05
   4.17966148e-05 3.41359555e-05 7.06000312e-04 1.92947628e-05
   4.74016233e-05 5.33839921e-05 2.16858243e-05 5.09072124e-05
   3.84136656e-05 4.94235683e-05 7.20680182e-05 1.75826644e-05
   1.11210888e-04 1.13037429e-04 1.09643348e-04 4.92015970e-05
   2.30149781e-05 5.09998317e-05 8.83653702e-05 6.67178392e-05
   3.35102186e-06 1.27836165e-06 1.49351690e-05 1.40283119e-05
   8.95756384e-05 1.54491081e-05 1.04128587e-04 1.66032914e-04
   1.47720057e-04 1.81505766e-05 6.44068086e-06 3.05069916e-05
   5.64049042e-05 1.59723004e-06 4.23822621e-06 1.75711157e-05
   1.56284441e-05 2.06641344e-04 8.10355923e-05]
  [1.15814304e-03 2.31297945e-05 4.79320579e-06 9.19726954e-05
   4.34800722e-06 7.80728783e-07 6.65805564e-05 4.89390914e-06
   2.54268853e-05 1.21166144e-04 1.30465355e-06 2.1

In [12]:
# Predicting Probability of Input Sentence

def find_probability(model_RNN, courpus_tokenizer, sentence):
    enter = courpus_tokenizer.texts_to_sequences([sentence])[0]
    enter.insert(0,0)
    enter = array(enter)
    enter = numpy.reshape(enter, newshape=(1,-1))
    print("Enter is : ", enter)
    print("Shape of enter is : ", enter.shape)
    probability = model_RNN.predict_proba(enter, verbose=0)
    probability_t = 1
    for i in range(probability.shape[1]-1):
        probability_t = probability_t * probability[0, i,
                                                      enter[0, i+1]]
    print("Probability of sentence - '{}' is {}".format(sentence, probability_t))

while 1:
    sentence = input(  "Please enter your sentence : " )
    find_probability(model, tk, sentence)
    choice = input(  "\nWould you like to continue? Enter N/n to stop : " ).lower()
    if choice == 'n':
        print("Done! It is completed!")
        break





Please enter your sentence : messi and ronaldo


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


Enter is :  [[ 0 28  7 29]]
Shape of enter is :  (1, 4)
Probability of sentence - 'messi and ronaldo' is 6.589099701933851e-15

Would you like to continue? Enter N/n to stop : no
Please enter your sentence : In modern Football messi
Enter is :  [[ 0  2  8  9 28]]
Shape of enter is :  (1, 5)
Probability of sentence - 'In modern Football messi' is 1.0793117142595449e-10

Would you like to continue? Enter N/n to stop : n
Done! It is completed!


In [15]:
# Sentance Generation
def sample_sequence_all(model_RNN, courpus_tokenizer, words, size_vocab):
  enter = list()
  text_input = ''
  for i in range(words):
    print("text_input being taken as '{}'".format(text_input))
    enter = courpus_tokenizer.texts_to_sequences([text_input])[0]
    enter.insert(0,0)
    enter = array(enter)
    enter = numpy.reshape(enter, newshape=(1,-1))
    print("For i : {} enter is : {}".format(i, enter))
    
    if i==0:
      probability = model_RNN.predict_proba(enter, verbose=0)
      y_predict = 0
      while y_predict==0:
        y_predict = numpy.random.choice(range(size_vocab), p = probability.ravel())
      y_predict = numpy.reshape(numpy.array([y_predict]), newshape = (1,-1))
      print("For i : {} y_predict in if is : {}".format(i, y_predict))
      
    
    else:
      probability = model_RNN.predict_proba(enter, verbose=0)
      y_predict = numpy.append(y_predict,0)
      y_predict = numpy.reshape(y_predict, newshape = (1,-1))

      while y_predict[0,i]==0:
        y_predict[0,i]=numpy.random.choice(range(size_vocab),p=probability[0,i].ravel())
      print("For i : {} y_predict in else is : {}".format(i, y_predict))

    
    exit = ''
    for w, index in courpus_tokenizer.word_index.items():
      if index == y_predict[0,i]:
        exit = w
        break
    text_input += exit + ' '
    print('\n' + '-'*50 + '\n')
  return text_input



totWords = int(input( "Enter number of total  words to be genrated : " ))
sample_sequence_all(model, tk, totWords , vocab_len)
    

Enter number of total  words to be genrated : 4
text_input being taken as ''
For i : 0 enter is : [[0]]
For i : 0 y_predict in if is : [[1]]

--------------------------------------------------

text_input being taken as 'the '
For i : 1 enter is : [[0 1]]
For i : 1 y_predict in else is : [[ 1 26]]

--------------------------------------------------

text_input being taken as 'the game '
For i : 2 enter is : [[ 0  1 26]]
For i : 2 y_predict in else is : [[ 1 26 27]]

--------------------------------------------------

text_input being taken as 'the game that '
For i : 3 enter is : [[ 0  1 26 27]]
For i : 3 y_predict in else is : [[ 1 26 27 28]]

--------------------------------------------------



/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:425: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


'the game that messi '